In [1]:
import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, auc_score
from scipy import sparse
from datetime import datetime
from catboost import CatBoostRegressor

C:\Users\Evgeny\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
movies_imdb = pd.read_csv('./movies_imdb.csv')

In [3]:
als_train_u0_m0 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_0_m_0.csv')
als_train_u0_m1 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_0_m_1.csv')
als_train_u0_m2 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_0_m_2.csv')
als_train_u1_m0 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_1_m_0.csv')
als_train_u1_m1 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_1_m_1.csv')
als_train_u1_m2 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_1_m_2.csv')
als_train_u2_m0 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_2_m_0.csv')
als_train_u2_m1 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_2_m_1.csv')
als_train_u2_m2 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_2_m_2.csv')

als_test = pd.read_csv('./ALS/ratings_dev_with_all_ALS_predictions.csv')
als_final_test = pd.read_csv('./ALS/ratings_test_with_all_ALS_predictions.csv')

In [4]:
a = pd.read_csv('./predictions_linear/predictions_0', header=None)

In [5]:
print(a.shape)
print(als_train_u0_m0.shape)
print(a.head(3))
print(als_train_u0_m0.head(3))

(1567557, 5)
(1567557, 7)
   0       1   2          3         4
0  0   53434  19  822873600  2.558093
1  0  124035  47  823185196  3.624575
2  0  124035  52  823185197  3.384482
   movieId  userId   timestamp  rating  user_movie_ALS  user_genre_ALS  \
0    33794  108543  1141315698     4.0        4.061448        3.435761   
1    33794  116656  1308349074     4.5        4.055828        3.234410   
2    33794  113116  1297394692     3.0        3.999297        3.608127   

   user_actor_ALS  
0        3.240306  
1        3.690494  
2        3.915428  


In [6]:
lin_features_0 = pd.read_csv('./predictions_linear/predictions_0', header=None)
lin_features_1 = pd.read_csv('./predictions_linear/predictions_1', header=None)
lin_features_2 = pd.read_csv('./predictions_linear/predictions_2', header=None)
lin_features_3 = pd.read_csv('./predictions_linear/predictions_3', header=None)
lin_features_4 = pd.read_csv('./predictions_linear/predictions_4', header=None)
lin_features_5 = pd.read_csv('./predictions_linear/predictions_5', header=None)
lin_features_6 = pd.read_csv('./predictions_linear/predictions_6', header=None)
lin_features_7 = pd.read_csv('./predictions_linear/predictions_7', header=None)
lin_features_8 = pd.read_csv('./predictions_linear/predictions_8', header=None)

In [7]:
lin_features_dev = pd.read_csv('./predictions_linear/dev_predicftions', header=None)
lin_features_final_test = pd.read_csv('./predictions_linear/test_predicftions', header=None)

In [8]:
lin_features_0.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_1.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_2.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_3.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_4.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_5.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_6.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_7.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_8.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)

In [9]:
lin_features_dev.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_final_test.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)

In [10]:
print(lin_features_1.shape)
print(als_train_u0_m1.shape)

(1544890, 5)
(1544890, 7)


In [11]:
lin_features_0.head(2)

,fold,userId,movieId,timestamp,lin_pred
0,0,53434,19,822873600,2.558093
1,0,124035,47,823185196,3.624575


In [12]:
als_train_u0_m0 = pd.merge(als_train_u0_m0, lin_features_0, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u0_m1 = pd.merge(als_train_u0_m1, lin_features_1, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u0_m2 = pd.merge(als_train_u0_m2, lin_features_2, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u1_m0 = pd.merge(als_train_u1_m0, lin_features_3, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u1_m1 = pd.merge(als_train_u1_m1, lin_features_4, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u1_m2 = pd.merge(als_train_u1_m2, lin_features_5, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u2_m0 = pd.merge(als_train_u2_m0, lin_features_6, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u2_m1 = pd.merge(als_train_u2_m1, lin_features_7, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u2_m2 = pd.merge(als_train_u2_m2, lin_features_8, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])

In [13]:
als_test = pd.merge(als_test, lin_features_dev, how='left', 
                    left_on=['userId', 'movieId', 'timestamp'], 
                    right_on=['userId', 'movieId', 'timestamp'])
als_final_test = pd.merge(als_final_test, lin_features_final_test, how='left', 
                          left_on=['userId', 'movieId', 'timestamp'], 
                          right_on=['userId', 'movieId', 'timestamp'])

In [14]:
print(lin_features_dev.shape)
print(als_test.shape)

(1690360, 5)
(1690360, 9)


In [15]:
print(lin_features_final_test.shape)
print(als_final_test.shape)

(3790860, 5)
(3790860, 8)


In [16]:
als_train_u1_m1.head(2)

,movieId,userId,timestamp,rating,user_movie_ALS,user_genre_ALS,user_actor_ALS,fold,lin_pred
0,6502,10416,1127117051,4.0,3.706461,3.368054,NaN,4,3.625825
1,24,23457,1205769764,5.0,3.173135,3.481067,3.033822,4,3.106729


In [17]:
als_test.head(2)

,userId,movieId,timestamp,rating,user_movie_ALS,user_genre_ALS,user_actor_ALS,fold,lin_pred
0,40965,1217,1180039623,5.0,3.616524,3.165045,3.187873,9,4.238109
1,131172,466,835337608,4.0,2.412944,3.676014,2.545692,9,3.021585


In [18]:
als_train = pd.concat([als_train_u0_m0, als_train_u0_m1, als_train_u0_m2,
                       als_train_u1_m0, als_train_u1_m1, als_train_u1_m2,
                       als_train_u2_m0, als_train_u2_m1, als_train_u2_m2],
                      ignore_index=True)

In [19]:
movies_imdb.head(3)

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_imdb,movieId,title,genres,titleCleaned,year,nconst,averageRating,numVotes
0,0,tt0066856,movie,born to win,born to win,0,1971,\N,88,"Comedy,Crime,Drama",3212,Born to Win (1971),Drama,born to win,1971,nm0005837nm0001719nm0696038nm0000947nm0282031n...,5.5,1432.0
1,1,tt0066858,movie,boy friend,boy friend,0,1971,\N,137,"Comedy,Musical,Romance",74859,"Boy Friend, The (1971)",Comedy|Musical|Romance,boy friend,1971,nm0910589nm0493341nm0300083nm0012451nm0697799n...,6.8,2764.0
2,2,tt0066892,movie,crnl knowledge,crnl knowledge,0,1971,\N,98,Drama,3167,Carnal Knowledge (1971),Comedy|Drama,crnl knowledge,1971,nm0000197nm0000298nm0000268nm0307316nm0001566n...,7.0,10324.0


In [20]:
print(movies_imdb.shape)
print(movies_imdb['movieId'].nunique())

(19636, 18)
19636


In [21]:
movies_imdb['averageRating'] = movies_imdb['averageRating'] / 2

In [22]:
genres = list(map(lambda x: [el for el in x.split(',')] + ['None'] * (3 - len(x.split(','))), movies_imdb['genres_imdb']))

In [23]:
movies_imdb['genres_imdb_0'] = [el[0] for el in genres]
movies_imdb['genres_imdb_1'] = [el[1] for el in genres]
movies_imdb['genres_imdb_2'] = [el[2] for el in genres]

In [24]:
sum(movies_imdb['year'].isna())

0

In [25]:
movies_imdb['runtimeMinutes'][movies_imdb['runtimeMinutes'] == '\\N'] = -1

C:\Users\Evgeny\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [26]:
movies_imdb.head(3)

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_imdb,...,title,genres,titleCleaned,year,nconst,averageRating,numVotes,genres_imdb_0,genres_imdb_1,genres_imdb_2
0,0,tt0066856,movie,born to win,born to win,0,1971,\N,88,"Comedy,Crime,Drama",...,Born to Win (1971),Drama,born to win,1971,nm0005837nm0001719nm0696038nm0000947nm0282031n...,2.75,1432.0,Comedy,Crime,Drama
1,1,tt0066858,movie,boy friend,boy friend,0,1971,\N,137,"Comedy,Musical,Romance",...,"Boy Friend, The (1971)",Comedy|Musical|Romance,boy friend,1971,nm0910589nm0493341nm0300083nm0012451nm0697799n...,3.40,2764.0,Comedy,Musical,Romance
2,2,tt0066892,movie,crnl knowledge,crnl knowledge,0,1971,\N,98,Drama,...,Carnal Knowledge (1971),Comedy|Drama,crnl knowledge,1971,nm0000197nm0000298nm0000268nm0307316nm0001566n...,3.50,10324.0,Drama,None,None


In [27]:
print(als_train.shape)
print(als_test.shape)
print(als_train.head(2))
print(als_test.head(2))
print(als_final_test.shape)
print(als_final_test.head(2))

(14453730, 9)
(1690360, 9)
   movieId  userId   timestamp  rating  user_movie_ALS  user_genre_ALS  \
0    33794  108543  1141315698     4.0        4.061448        3.435761   
1    33794  116656  1308349074     4.5        4.055828        3.234410   

   user_actor_ALS  fold  lin_pred  
0        3.240306     0  3.770295  
1        3.690494     0  3.869308  
   userId  movieId   timestamp  rating  user_movie_ALS  user_genre_ALS  \
0   40965     1217  1180039623     5.0        3.616524        3.165045   
1  131172      466   835337608     4.0        2.412944        3.676014   

   user_actor_ALS  fold  lin_pred  
0        3.187873     9  4.238109  
1        2.545692     9  3.021585  
(3790860, 8)
   userId  movieId   timestamp  user_movie_ALS  user_genre_ALS  \
0    2711     5902  1044313557        4.242603        3.521633   
1    3352     1639   895849643        3.059602        3.224138   

   user_actor_ALS  fold  lin_pred  
0             NaN    10  3.643722  
1        3.300966    10  2.

In [28]:
def rmse_asymmetric(rating, prediction):
    bad_border = 4
    
    if rating < bad_border <= prediction:
        penalty_coefficient = 2.0
    else:
        penalty_coefficient = 1.0
    
    return penalty_coefficient*(rating - prediction)**2


def vector_rmse_asymmetric(df, true_col, pred_col, bad_border=4):
    data = np.array(df[[true_col, pred_col]].values)
    mask_1 = data[:, 0] < bad_border
    mask_2 = data[:, 1] >= bad_border
    penalty = data[mask_1 & mask_2]
    not_penalty = data[~mask_1 & mask_2]
    result = np.mean(np.concatenate((2 * (penalty[:, 0] - penalty[:, 1])  ** 2,
                                    (not_penalty[:, 0] - not_penalty[:, 1])  ** 2), 
                                    axis=None))
    return result

In [29]:
als_train = pd.merge(als_train, movies_imdb, how='left', 
                     left_on=['movieId'], right_on=['movieId'])
als_test = pd.merge(als_test, movies_imdb, how='left', 
                     left_on=['movieId'], right_on=['movieId'])
als_final_test = pd.merge(als_final_test, movies_imdb, how='left', 
                     left_on=['movieId'], right_on=['movieId'])

In [30]:
als_train['titleType'].fillna('None', inplace=True)
als_test['titleType'].fillna('None', inplace=True)
als_final_test['titleType'].fillna('None', inplace=True)

als_train['genres_imdb_0'].fillna('None', inplace=True)
als_test['genres_imdb_0'].fillna('None', inplace=True)
als_final_test['genres_imdb_0'].fillna('None', inplace=True)

als_train['genres_imdb_1'].fillna('None', inplace=True)
als_test['genres_imdb_1'].fillna('None', inplace=True)
als_final_test['genres_imdb_1'].fillna('None', inplace=True)

als_train['genres_imdb_2'].fillna('None', inplace=True)
als_test['genres_imdb_2'].fillna('None', inplace=True)
als_final_test['genres_imdb_2'].fillna('None', inplace=True)

In [31]:
als_train.head(3)

,movieId,userId,timestamp,rating,user_movie_ALS,user_genre_ALS,user_actor_ALS,fold,lin_pred,Unnamed: 0,...,title,genres,titleCleaned,year,nconst,averageRating,numVotes,genres_imdb_0,genres_imdb_1,genres_imdb_2
0,33794,108543,1141315698,4.0,4.061448,3.435761,3.240306,0,3.770295,9943.0,...,Batman Begins (2005),Action|Crime|IMAX,btmn begins,2005.0,nm0746273nm0000288nm0000323nm0913822nm0000553n...,4.1,1224005.0,Action,Adventure,None
1,33794,116656,1308349074,4.5,4.055828,3.234410,3.690494,0,3.869308,9943.0,...,Batman Begins (2005),Action|Crime|IMAX,btmn begins,2005.0,nm0746273nm0000288nm0000323nm0913822nm0000553n...,4.1,1224005.0,Action,Adventure,None
2,33794,113116,1297394692,3.0,3.999297,3.608127,3.915428,0,3.770065,9943.0,...,Batman Begins (2005),Action|Crime|IMAX,btmn begins,2005.0,nm0746273nm0000288nm0000323nm0913822nm0000553n...,4.1,1224005.0,Action,Adventure,None


In [32]:
als_test.columns

Index(['userId', 'movieId', 'timestamp', 'rating', 'user_movie_ALS',
       'user_genre_ALS', 'user_actor_ALS', 'fold', 'lin_pred', 'Unnamed: 0',
       'tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres_imdb', 'title',
       'genres', 'titleCleaned', 'year', 'nconst', 'averageRating', 'numVotes',
       'genres_imdb_0', 'genres_imdb_1', 'genres_imdb_2'],
      dtype='object')

In [33]:
als_train['curr_date'] = list(map(lambda x: datetime.fromtimestamp(x).isoformat(), als_train['timestamp']))
als_test['curr_date'] = list(map(lambda x: datetime.fromtimestamp(x).isoformat(), als_test['timestamp']))
als_final_test['curr_date'] = list(map(lambda x: datetime.fromtimestamp(x).isoformat(), als_final_test['timestamp']))

In [34]:
als_train['curr_year'] = list(map(lambda x: float(x[:4]), als_train['curr_date']))
als_test['curr_year'] = list(map(lambda x: float(x[:4]), als_test['curr_date']))
als_final_test['curr_year'] = list(map(lambda x: float(x[:4]), als_final_test['curr_date']))

als_train['curr_month'] = list(map(lambda x: x[5:7], als_train['curr_date']))
als_test['curr_month'] = list(map(lambda x: x[5:7], als_test['curr_date']))
als_final_test['curr_month'] = list(map(lambda x: x[5:7], als_final_test['curr_date']))

als_train['movie_age'] = als_train['curr_year'] - als_train['year']
als_test['movie_age'] = als_test['curr_year'] - als_test['year']
als_final_test['movie_age'] = als_final_test['curr_year'] - als_final_test['year']

In [35]:
als_train.head(1)

,movieId,userId,timestamp,rating,user_movie_ALS,user_genre_ALS,user_actor_ALS,fold,lin_pred,Unnamed: 0,...,nconst,averageRating,numVotes,genres_imdb_0,genres_imdb_1,genres_imdb_2,curr_date,curr_year,curr_month,movie_age
0,33794,108543,1141315698,4.0,4.061448,3.435761,3.240306,0,3.770295,9943.0,...,nm0746273nm0000288nm0000323nm0913822nm0000553n...,4.1,1224005.0,Action,Adventure,None,2006-03-02T19:08:18,2006.0,03,1.0


In [36]:
print(vector_rmse_asymmetric(als_train, 'rating', 'user_movie_ALS'))
print(vector_rmse_asymmetric(als_train, 'rating', 'user_genre_ALS'))
print(vector_rmse_asymmetric(als_train, 'rating', 'user_actor_ALS'))
print(vector_rmse_asymmetric(als_train, 'rating', 'averageRating'))
print(vector_rmse_asymmetric(als_train, 'rating', 'lin_pred'))

C:\Users\Evgeny\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in greater_equal
  from ipykernel import kernelapp as app


0.6197099090875856
1.0423400236647804
0.7162128806061265
1.3887644861972723
1.4576650118188108


In [37]:
feature_columns = [
    'user_movie_ALS', 
    'user_genre_ALS', 
    'user_actor_ALS', 
    'averageRating', 
    'isAdult', 
    'runtimeMinutes', 
    'numVotes', 
    #'year', 
    'movie_age', 
    #'curr_year', 
    'lin_pred',
]
cat_feature_columns = [
    #'titleType', 
    #'genres_imdb_0', 
    #'genres_imdb_1', 
    #'genres_imdb_2', 
    #'curr_month'
]
train_columns = feature_columns + cat_feature_columns

In [38]:
train_data = als_train[train_columns]
train_target = als_train['rating']

test_data = als_test[train_columns]
test_target = als_test['rating']

In [39]:
bad_border = 4
mask_1 = np.array(train_target.values) < bad_border
weights = np.ones(len(train_data))
weights[mask_1] = 2

In [40]:
weights

array([1., 1., 2., ..., 2., 1., 2.])

In [41]:
model = CatBoostRegressor(iterations=500)

In [42]:
model.fit(train_data, train_target, verbose=True, sample_weight=weights, cat_features=cat_feature_columns)

0:	learn: 3.3413433	total: 1.41s	remaining: 11m 45s
1:	learn: 3.2471486	total: 2.77s	remaining: 11m 30s
2:	learn: 3.1561342	total: 4.16s	remaining: 11m 29s
3:	learn: 3.0678402	total: 5.3s	remaining: 10m 57s
4:	learn: 2.9823558	total: 6.47s	remaining: 10m 40s
5:	learn: 2.8996293	total: 7.61s	remaining: 10m 26s
6:	learn: 2.8196153	total: 8.76s	remaining: 10m 16s
7:	learn: 2.7421140	total: 10s	remaining: 10m 16s
8:	learn: 2.6671671	total: 11.2s	remaining: 10m 13s
9:	learn: 2.5948145	total: 12.5s	remaining: 10m 10s
10:	learn: 2.5246935	total: 13.7s	remaining: 10m 9s
11:	learn: 2.4569703	total: 14.9s	remaining: 10m 4s
12:	learn: 2.3913264	total: 16s	remaining: 9m 59s
13:	learn: 2.3278895	total: 17.2s	remaining: 9m 55s
14:	learn: 2.2666371	total: 18.2s	remaining: 9m 49s
15:	learn: 2.2072950	total: 19.3s	remaining: 9m 44s
16:	learn: 2.1500261	total: 20.4s	remaining: 9m 40s
17:	learn: 2.0946856	total: 21.6s	remaining: 9m 38s
18:	learn: 2.0413246	total: 22.7s	remaining: 9m 34s
19:	learn: 1.9897

156:	learn: 0.7952494	total: 2m 57s	remaining: 6m 28s
157:	learn: 0.7952031	total: 2m 58s	remaining: 6m 27s
158:	learn: 0.7951585	total: 3m	remaining: 6m 26s
159:	learn: 0.7951145	total: 3m 1s	remaining: 6m 24s
160:	learn: 0.7950761	total: 3m 2s	remaining: 6m 23s
161:	learn: 0.7950340	total: 3m 3s	remaining: 6m 22s
162:	learn: 0.7949939	total: 3m 4s	remaining: 6m 21s
163:	learn: 0.7949539	total: 3m 5s	remaining: 6m 20s
164:	learn: 0.7949213	total: 3m 6s	remaining: 6m 19s
165:	learn: 0.7948899	total: 3m 7s	remaining: 6m 18s
166:	learn: 0.7948603	total: 3m 9s	remaining: 6m 16s
167:	learn: 0.7948307	total: 3m 10s	remaining: 6m 15s
168:	learn: 0.7947994	total: 3m 11s	remaining: 6m 14s
169:	learn: 0.7947695	total: 3m 12s	remaining: 6m 13s
170:	learn: 0.7947401	total: 3m 13s	remaining: 6m 12s
171:	learn: 0.7947111	total: 3m 14s	remaining: 6m 11s
172:	learn: 0.7946821	total: 3m 15s	remaining: 6m 10s
173:	learn: 0.7946552	total: 3m 17s	remaining: 6m 9s
174:	learn: 0.7946301	total: 3m 18s	remai

310:	learn: 0.7933419	total: 6m	remaining: 3m 38s
311:	learn: 0.7933349	total: 6m 1s	remaining: 3m 37s
312:	learn: 0.7933284	total: 6m 2s	remaining: 3m 36s
313:	learn: 0.7933233	total: 6m 3s	remaining: 3m 35s
314:	learn: 0.7933175	total: 6m 5s	remaining: 3m 34s
315:	learn: 0.7933121	total: 6m 6s	remaining: 3m 33s
316:	learn: 0.7933063	total: 6m 7s	remaining: 3m 32s
317:	learn: 0.7933006	total: 6m 8s	remaining: 3m 30s
318:	learn: 0.7932952	total: 6m 9s	remaining: 3m 29s
319:	learn: 0.7932893	total: 6m 10s	remaining: 3m 28s
320:	learn: 0.7932833	total: 6m 11s	remaining: 3m 27s
321:	learn: 0.7932778	total: 6m 12s	remaining: 3m 26s
322:	learn: 0.7932725	total: 6m 14s	remaining: 3m 25s
323:	learn: 0.7932676	total: 6m 15s	remaining: 3m 23s
324:	learn: 0.7932616	total: 6m 16s	remaining: 3m 22s
325:	learn: 0.7932569	total: 6m 17s	remaining: 3m 21s
326:	learn: 0.7932526	total: 6m 18s	remaining: 3m 20s
327:	learn: 0.7932471	total: 6m 20s	remaining: 3m 19s
328:	learn: 0.7932423	total: 6m 21s	rema

464:	learn: 0.7927045	total: 8m 58s	remaining: 40.6s
465:	learn: 0.7927025	total: 8m 59s	remaining: 39.4s
466:	learn: 0.7926994	total: 9m	remaining: 38.2s
467:	learn: 0.7926963	total: 9m 2s	remaining: 37.1s
468:	learn: 0.7926924	total: 9m 3s	remaining: 35.9s
469:	learn: 0.7926904	total: 9m 4s	remaining: 34.7s
470:	learn: 0.7926874	total: 9m 5s	remaining: 33.6s
471:	learn: 0.7926854	total: 9m 6s	remaining: 32.4s
472:	learn: 0.7926825	total: 9m 7s	remaining: 31.3s
473:	learn: 0.7926793	total: 9m 8s	remaining: 30.1s
474:	learn: 0.7926752	total: 9m 10s	remaining: 28.9s
475:	learn: 0.7926724	total: 9m 11s	remaining: 27.8s
476:	learn: 0.7926695	total: 9m 12s	remaining: 26.6s
477:	learn: 0.7926670	total: 9m 13s	remaining: 25.5s
478:	learn: 0.7926643	total: 9m 14s	remaining: 24.3s
479:	learn: 0.7926585	total: 9m 15s	remaining: 23.2s
480:	learn: 0.7926538	total: 9m 16s	remaining: 22s
481:	learn: 0.7926503	total: 9m 17s	remaining: 20.8s
482:	learn: 0.7926450	total: 9m 19s	remaining: 19.7s
483:	l

In [43]:
catboost_predict = model.predict(test_data)

In [44]:
als_test['catboost_predict'] = catboost_predict

In [45]:
print(vector_rmse_asymmetric(als_test, 'rating', 'catboost_predict'))

0.6075941655183583


In [46]:
model.feature_importances_

array([9.22274846e+01, 1.41638014e+00, 6.23260480e-01, 1.01311962e+00,
       3.11990892e-02, 1.81897937e-01, 8.94276363e-01, 5.98892820e-01,
       3.01348894e+00])

In [84]:
als_test['mean_predict'] = np.mean(test_data[['user_movie_ALS', 'user_genre_ALS', 
                                              'user_actor_ALS', 'averageRating', 'lin_pred']].values, axis=1)

In [85]:
print(vector_rmse_asymmetric(als_test, 'rating', 'mean_predict'))

0.6360877513509919


C:\Users\Evgeny\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in greater_equal
  from ipykernel import kernelapp as app


In [86]:
als_test['median_predict'] = np.median(test_data[['user_movie_ALS', 'user_genre_ALS', 
                                                  'user_actor_ALS', 'averageRating', 'lin_pred']].values, axis=1)

C:\Users\Evgeny\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3405: RuntimeWarning: Invalid value encountered in median for 671110 results
  r = func(a, **kwargs)


In [87]:
print(vector_rmse_asymmetric(als_test, 'rating', 'median_predict'))

0.6469539427984751


C:\Users\Evgeny\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in greater_equal
  from ipykernel import kernelapp as app


In [88]:
final_test_results = model.predict(als_final_test[train_columns])

In [89]:
als_final_test['prediction'] = final_test_results

In [90]:
als_final_test.to_csv('./prediction_4.csv')

In [91]:
import pickle

In [92]:
with open('./saved_catboost_model_4.pkl', 'wb') as f:
    pickle.dump(model, f)

In [93]:
als_final_test.head(5)

,userId,movieId,timestamp,user_movie_ALS,user_genre_ALS,user_actor_ALS,fold,lin_pred,Unnamed: 0,tconst,...,averageRating,numVotes,genres_imdb_0,genres_imdb_1,genres_imdb_2,curr_date,curr_year,curr_month,movie_age,prediction
0,2711,5902,1044313557,4.242603,3.521633,NaN,10,3.643722,NaN,NaN,...,NaN,NaN,None,None,None,2003-02-04T02:05:57,2003.0,02,NaN,4.178553
1,3352,1639,895849643,3.059602,3.224138,3.300966,10,2.878951,5622.0,tt0118842,...,3.60,127000.0,Comedy,Drama,Romance,1998-05-22T18:07:23,1998.0,05,1.0,2.987103
2,3704,3646,1048098193,2.311651,3.442956,3.009916,10,4.311557,7473.0,tt0208003,...,2.55,77266.0,Action,Comedy,Crime,2003-03-19T21:23:13,2003.0,03,3.0,2.291752
3,4391,63876,1231089682,3.963379,3.620384,3.963878,10,3.573920,14474.0,tt1013753,...,3.75,157885.0,Biography,Drama,None,2009-01-04T20:21:22,2009.0,01,1.0,3.865707
4,7850,3094,960252000,4.571631,4.097393,4.220759,10,4.916982,2527.0,tt0093512,...,3.85,14533.0,Drama,Romance,None,2000-06-06T03:40:00,2000.0,06,13.0,4.582771
